In [1]:
import pymongo
import pickle
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
import re
import csv

import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS

In [2]:
def cleaner(string):
    string = re.sub('http*\S*', '', string)
    string = re.sub('amp;', '', string)
    string = re.sub('\n', ' ', string)
    return string

In [14]:
alltweets_df = pd.read_csv('tweets.csv', names=['user_name', 'tw_id', 'text'], encoding='utf-8')

In [15]:
alltweets_df['tw_id'].count()

1405833

In [16]:
# drop duplicates on tw_id and text
alltweets_df = alltweets_df.drop_duplicates('tw_id')

In [17]:
alltweets_df['tw_id'].count()

1366704

In [18]:
alltweets_df = alltweets_df.drop_duplicates('text')

In [19]:
alltweets_df['tw_id'].count()

1286437

In [20]:
# drop retweets
alltweets_df = alltweets_df[~alltweets_df['text'].str.startswith('RT')]

In [21]:
alltweets_df['tw_id'].count()

901873

In [22]:
alltweets_df.sample(40)

,user_name,tw_id,text
205533,dbasolved,872447708534984704,why doesn’t my code formatter work on my blog now…..grrr #tryingtobeproductive #newpostcoming
362534,bubzTheSensei,771821061092409346,Pablo Escobar. Top Gangster in a Netflix Original. #NetflixOG
912450,KemiEgan,872731937483497473,ABS Capital strives to provide YOU a personalised &amp; dedicated #service of your #investment process and beyond.… https://t.co/x3Wk0YEL7d
433902,MikeTNelson,870008084239962112,Advanced HRV Questions from a reader that I wanted to share here with all of you\n\nQuestions\nDo you have a... https://t.co/mxW1jFv7Iq
369135,CarstenGauslow,718481237262921728,Priorities lol
824285,jrycraft01,580517628084641792,FSRT course scenarios in the pool tonight-consolidating understanding. Great solutions to some sticky situations. Cracking #focus all round.
785424,luchanik,769802336499675138,Christmas At The Venetian https://t.co/543IwWnHT2 via @wordpressdotcom
1044381,tomnevakshonoff,663844996890558464,Our NDP government is going to work with partners in the community to ensure the sustainability of our fisheries.\nhttps://t.co/XYVI2VA1z5
1285611,ShopHeirlooms,870449400287567872,#LiveFromThe55 I'm so hyped b! I hope it's a good series! Ahhhhh! #NBAFinals https://t.co/witrfnPOxT
289096,LOLOTheFox,725414620882558976,These Wedding-Related Plans Have Been Linked To Divorce: Maybe having a small wedding isn't such a good idea ... https://t.co/Pu10cec0w1


In [23]:
# clean text
alltweets_df['text'] = alltweets_df['text'].apply(cleaner)

In [25]:
alltweets_df.sample(50)

,user_name,tw_id,text
596551,JJDacotah,869750523276263427,Fargo Invaders Inside the Play - Week 2 vs Warriors (W 14-9) via @YouTube
115936,photosbyjwarb,873974872757633025,"@Mackcinn Cheers, might follow suit shortly"
214709,RealJohnAbbott,870798430188310528,#SopranosSongs Bawitdabada-Bing @efoxband
549444,Vikaslamba19,793191783702204416,How One Successful Digital Entrepreneur Stays Entertained by Her Business: Sarah Morgan may rub some people t...
611582,Connor_Brennan1,714208889818980353,Just want to spend it all with my loyal chick
256861,glenysfurness,873289177139470337,@Willbomb90 Oh no :( - get well soon
389795,davyc1976,864551866369290240,@mowen23 @TwoLPhill Eastleigh pay daft money. Leave.em to.it I say. Holllands is no great loss
448199,DebraFoxx,847175426900418560,#SteveHarvey teaching 11/12yo boys about french kissing on today's show👍 or 👎 #terribleidea #worldgonemad #terribleshowtopic #why
291841,lills_baby,766499500500418560,I knew I wasn't the only one. It be relaxing..
80119,iPaul34,872158487664623616,@csav55 But has @clairesavjourno 🤔 lol


In [ ]:
# Drop non-English tweets - HOW?

def drop_non_english(string):
    alpha_emojis = ['a', 'b', 'c', 'd', ]
    for letter in string:
        

In [32]:
alltweets_df.sort('text', ascending=False)[:150]

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,user_name,tw_id,text,gender
464610,aRmeni7,728346123820249089,"󾮗🏻󾮗🏻iroq sa shedevr e, mer oreri shedevr",male
566139,BlackMtnCBGs,709189912063397890,󾮗,male
758220,robrichley75,771400102464323585,󾭞,male
347162,colleen_hoffman,747143880601047040,"󾭚Sunday Story Time󾭚 ""My name is Maegan. I am 21 years old and from a town of 450 people in southern Illinois. I...",female
347161,colleen_hoffman,747143880601047040,"󾭚Sunday Story Time󾭚 ""My name is Maegan. I am 21 years old and from a town of 450 people in southern Illinois. I...",female
347353,colleen_hoffman,744923786521116672,󾭚LIMITED TIME OFFER󾭚 I have 󾠻 FREE boxes of wraps to GIVE AWAY to the next 󾠻 People who join our 12 Week 12 Wrap...,female
347354,colleen_hoffman,744923786521116672,󾭚LIMITED TIME OFFER󾭚 I have 󾠻 FREE boxes of wraps to GIVE AWAY to the next 󾠻 People who join our 12 Week 12 Wrap...,female
481901,MaxineMusic,706543879617548288,󾬙,female
635257,LisaWeidenbach,705974932426063872,󾬖󾬖󾬖,female
252380,CaliSkinArt,748966903587049472,󾬖,female


In [29]:
gender_df = pd.read_pickle('crowd_flower_gender_df')

In [30]:
# add gender column to tweet_df, joining on 'user_name'
alltweets_df = pd.merge(alltweets_df, gender_df, how='left', on='user_name')

In [31]:
alltweets_df.sample(10)

,user_name,tw_id,text,gender
493869,poetniall,815508633752637440,though the former is probably good advice to a mid career spoken worder...,male
859057,GraceMartin51,792148501203263489,Bob Dylan finally breaks his silence on Nobel Prize:,male
73988,SWaltersTV,807618479280881665,Breakfast with Santa! 🎅 @ Lakeway Activity Center,female
146598,terralightfoot,837177922381299712,32nd Anniversary of Spinal Tap today. Probably my favorite movie ever and so this is a good day to be alive ⭐️⭐️⭐️…,female
776860,GiveawayLovers,821003160022056961,"When you’ve got a cold, how does accomplishing a drama- make you feel? #sponsored #AdvilSinusCrowd",female
347474,MisisipJns,859972201319190528,#walrus #king #montreal #streetart #mural,female
666425,ralphsutton,723643905971900418,"Recording at #PasileyParkStudios with #Prince was a great privilege, I am thankful for the time spent.#RestInPower",male
68767,JerryHarte,867605147320438785,What do you do when she says a motorcycle is not a good vehicle for the drive in theater :P,male
153316,rubakaay,829462648408387584,@WokeSalam smh so ungrateful,female
37223,chipndale_,871741341285404672,@ItsNot_BRANDON Weirdest thing I've seen on this app to date lol,male


In [33]:
alltweets_df['gender'].value_counts()

male       489425
female     448778
brand      1683  
unknown    809   
Name: gender, dtype: int64

In [34]:
# drop brand and unknown
m_f_tweets_df = alltweets_df[(alltweets_df['gender']=='male') | (alltweets_df['gender']=='female')]

In [35]:
m_f_tweets_df['gender'].value_counts()

male      489425
female    448778
Name: gender, dtype: int64

In [36]:
# pickle the df
m_f_tweets_df.to_pickle('m_f_tweets_df')